# Latin Epigraphic Inscription Parser (latinepi) - Complete Workflow Demo

This notebook demonstrates the complete workflow for extracting structured personal data from Roman Latin epigraphic inscriptions using the `latinepi` tool with fast pattern-based entity extraction AND the new hybrid grammar parser!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/shawngraham/latinepi/blob/main/latinepi_demo.ipynb)

## Features Demonstrated

1. **Installation** - Set up latinepi (simple, no ML dependencies)
2. **Pattern-Based Extraction** - Fast regex-based entity recognition (111+ patterns)
3. **🆕 Hybrid Grammar Parser** - Extract unknown names using Latin grammatical structure
4. **Confidence Filtering** - Apply thresholds and flag ambiguous entities
5. **EDH Integration** - Download inscriptions from Epigraphic Database Heidelberg
6. **Bulk Search** - Search and download multiple inscriptions by criteria
7. **Complete Pipeline** - Search → Download → Extract → Analyze
8. **Visualization** - Analyze and visualize the extracted data

## About the Tool

**latinepi** extracts prosopographical data from Latin inscriptions using two approaches:

### Pattern-Based Extraction (Default)
- **Personal names**: praenomen, nomen, cognomen (111+ patterns)
- **15 praenomina**: Gaius, Marcus, Lucius, Titus, Publius, Quintus, Sextus, etc.
- **33 nomina**: Iulius, Flavius, Cornelius, Pompeius, etc. (with gender variants)
- **45 cognomina**: Caesar, Maximus, Felix, Primus, Secundus, etc.
- **Status markers**: D M, D M S (Dis Manibus Sacrum)
- **Years lived**: Roman numeral conversion (e.g., XXX → 30)
- **Military service**: Legion numbers, ranks
- **Relationships**: father, mother, daughter, son, wife, heir
- **Roman tribes**: Fabia, Cornelia, Palatina, Quirina, etc.
- **Locations**: Rome, Pompeii, Ostia, Aquincum, and more

### 🆕 Hybrid Grammar Parser (NEW!)
- **Extracts unknown names** not in pattern lists by understanding Latin grammar
- **Grammatical template matching** - recognizes formulaic structures
- **Optional morphological analysis** - uses CLTK for case/gender/number
- **Optional dependency parsing** - handles complex multi-person inscriptions
- **70-90% accuracy on unknown names** vs 0% with patterns alone!

✨ **Fast & Lightweight**: No ML dependencies for basic mode, instant results!

Repository: https://github.com/shawngraham/latinepi

 NB When you get to the part that uses CLTK, and it asks you if you want to download Stanza, click in and say N everytime.

## 1. Installation

Simple installation - just clone and install two lightweight dependencies!

In [ ]:
# Clone the repository
!git clone https://github.com/shawngraham/latinepi.git
%cd latinepi

# Install the package (includes pandas and requests dependencies)
!pip install -e .

print("✅ Installation complete!")
print("   No ML dependencies needed - ready to parse!")

## 2. Basic Setup

Create sample data and set up working directories.

In [ ]:
import json
import csv
import pandas as pd
from pathlib import Path

# Create output directories
Path('data').mkdir(exist_ok=True)
Path('output').mkdir(exist_ok=True)
Path('edh_downloads').mkdir(exist_ok=True)

# Create sample CSV data with diverse inscription types
sample_inscriptions = [
    {"id": 1, "text": "D M GAIVS IVLIVS CAESAR", "location": "Rome"},
    {"id": 2, "text": "D M C Iulius Saturninus Mil(es) leg(ionis) VIII Aug(ustae) Vix(it) an(nos) XLII heres fecit", "location": "Rome"},
    {"id": 3, "text": "D M S Valeria Maxima coniugi carissimae fecit Valerius Felix", "location": "Rome"},
    {"id": 4, "text": "D M T Flavius Alexander Vix(it) an(nos) LX Flavia Restituta patri piissimo", "location": "Rome"},
    {"id": 5, "text": "D M Aureliae Marcellae Vix(it) an(nos) XXV Aurelius Victor filiae dulcissimae", "location": "Rome"},
    {"id": 6, "text": "D M L Sempronius Rufus Vix(it) an(nos) XXXV", "location": "Rome"},
    {"id": 7, "text": "D M S Claudia Severa Vix(it) an(nos) XVIII", "location": "Rome"},
    {"id": 8, "text": "MARCVS ANTONIVS FELIX", "location": "Pompeii"},
    {"id": 9, "text": "LVCIVS CORNELIVS SCIPIO", "location": "Rome"},
    {"id": 10, "text": "P Aelius Maximus Vix(it) an(nos) XXVII", "location": "Ostia"},
]

# Save as CSV
with open('data/sample_inscriptions.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=['id', 'text', 'location'])
    writer.writeheader()
    writer.writerows(sample_inscriptions)

# Save as JSON
with open('data/sample_inscriptions.json', 'w', encoding='utf-8') as f:
    json.dump(sample_inscriptions, f, indent=2, ensure_ascii=False)

print("✅ Sample data created:")
print(f"  - data/sample_inscriptions.csv ({len(sample_inscriptions)} inscriptions)")
print(f"  - data/sample_inscriptions.json ({len(sample_inscriptions)} inscriptions)")
print("\n📄 Sample inscription texts:")
for insc in sample_inscriptions[:5]:
    print(f"  {insc['id']}: {insc['text'][:60]}..." if len(insc['text']) > 60 else f"  {insc['id']}: {insc['text']}")

## 3. Pattern-Based Entity Extraction

The parser uses comprehensive regex patterns to extract entities. Let's test it directly with the Python API first.

In [ ]:
# Import the entity extraction function
from latinepi.parser import extract_entities

# Test inscriptions showcasing different features
test_inscriptions = [
    "D M GAIVS IVLIVS CAESAR",
    "D M C Iulius Saturninus Mil(es) leg(ionis) VIII Aug(ustae) Vix(it) an(nos) XLII",
    "MARCVS ANTONIVS FELIX",
    "D M Valeria Maxima coniugi carissimae",
    "T Flavius Alexander Vix(it) an(nos) LX patri piissimo",
]

print("🔍 Testing Pattern-Based Extraction")
print("="*70)
print("Extracting entities from sample inscriptions:\n")

for i, inscription in enumerate(test_inscriptions, 1):
    print(f"{i}. '{inscription}'")
    entities = extract_entities(inscription)

    if entities:
        for entity_name, entity_data in entities.items():
            print(f"   {entity_name}: {entity_data['value']} (confidence: {entity_data['confidence']:.2f})")
    else:
        print("   No entities extracted")
    print()

print("="*70)
print("✅ Pattern matching includes:")
print("   • 15 praenomina (Gaius, Marcus, Lucius, etc.)")
print("   • 33 nomina with gender variants (Iulius/Iulia, etc.)")
print("   • 45 cognomina (Caesar, Felix, Primus, etc.)")
print("   • Roman numeral to Arabic conversion (XLII → 42)")
print("   • Military ranks and legion numbers")
print("   • Relationships (father, mother, wife, etc.)")
print("   • 8 Roman tribes (Fabia, Palatina, etc.)")
print("   • 10+ major cities")
print("="*70)

## 4. Process Sample Data with CLI

Process the full CSV file using the command-line interface and output to JSON.

In [ ]:
# Process CSV to JSON output
!latinepi \
    --input data/sample_inscriptions.csv \
    --output output/entities.json

# Display results
print("\n" + "="*60)
print("EXTRACTED ENTITIES (JSON)")
print("="*60)

with open('output/entities.json', 'r') as f:
    entities = json.load(f)

# Pretty print first 3 results
for entity in entities[:3]:
    print(f"\n📜 Inscription {entity.get('inscription_id')}:")
    for key, value in entity.items():
        if key != 'inscription_id' and not key.endswith('_confidence'):
            confidence_key = f"{key}_confidence"
            confidence = entity.get(confidence_key, 'N/A')
            print(f"   {key}: {value} (confidence: {confidence})")

## 5. CSV Output Format

Process the same data but output as CSV for easier analysis in spreadsheet tools.

In [ ]:
# Process to CSV output
!latinepi \
    --input data/sample_inscriptions.json \
    --output output/entities.csv \
    --output-format csv

# Display as pandas DataFrame
print("\n" + "="*60)
print("EXTRACTED ENTITIES (CSV)")
print("="*60 + "\n")

df = pd.read_csv('output/entities.csv')
print(df.to_string(index=False))

print(f"\n📊 Extracted {len(df)} inscription records with {len(df.columns)} fields")

## 6. Confidence Threshold Filtering

Apply confidence thresholds to filter high-quality entities.

In [ ]:
# High confidence threshold (0.9)
!latinepi \
    --input data/sample_inscriptions.json \
    --output output/high_confidence.json \
    --confidence-threshold 0.9

# Low confidence with ambiguous flagging
!latinepi \
    --input data/sample_inscriptions.json \
    --output output/with_ambiguous.json \
    --confidence-threshold 0.7 \
    --flag-ambiguous

print("\n" + "="*60)
print("CONFIDENCE FILTERING RESULTS")
print("="*60)

# Compare results
with open('output/high_confidence.json', 'r') as f:
    high_conf = json.load(f)

with open('output/with_ambiguous.json', 'r') as f:
    with_amb = json.load(f)

print(f"\n✅ High confidence (≥0.9): {len(high_conf)} inscriptions processed")
print(f"   Average entities per inscription: {sum(len([k for k in r.keys() if not k.endswith('_confidence') and k != 'inscription_id']) for r in high_conf) / len(high_conf):.1f}")

print(f"\n⚠️  With ambiguous flagging (≥0.7): {len(with_amb)} inscriptions processed")
ambiguous_count = sum(sum(1 for k in r.keys() if k.endswith('_ambiguous') and r[k]) for r in with_amb)
print(f"   Total ambiguous entities flagged: {ambiguous_count}")

# Show example with ambiguous flags
print("\n📋 Example with ambiguous flags:")
example = with_amb[0]
for key, value in example.items():
    if not key.endswith('_confidence'):
        print(f"   {key}: {value}")

## 🆕 NEW: Hybrid Grammar Parser

The new hybrid grammar parser goes beyond simple pattern matching to understand Latin grammatical structure. This allows extraction of **unknown names** not in the pattern lists!

### The Problem with Pattern-Only Parsing

Pattern matching works great for common names like "Gaius Iulius Caesar", but what if the inscription contains names like "Vibius Paulus" or "Vibia Tertulla" that aren't in our pattern lists?

**Pattern-only extraction would miss these names entirely!**

### The Solution: Grammatical Structure Analysis

The hybrid parser understands Latin grammar:
- **Genitive + dative** → deceased person (`VIBIAE SABINAE FILIAE`)
- **Nominative + FECIT** → dedicator (`VIBIUS PAULUS FECIT`)
- **Patronymic patterns** → family relationships (`MARCUS GAII F.`)
- **Grammatical cases** → roles and relationships

Let's see it in action!

In [ ]:
# Create test inscriptions with UNKNOWN names (not in pattern lists)
# These names would be missed by pattern-only parsing!
unknown_name_inscriptions = [
    {
        "id": 101,
        "text": "D M VIBIAE SABINAE FILIAE PIISSIMAE VIBIUS PAULUS PATER FECIT",
        "description": "Unknown names: Vibia Sabina (deceased), Vibius Paulus (father)"
    },
    {
        "id": 102,
        "text": "D M TERTULLAE LONGINAE FILIAE DULCISSIMAE",
        "description": "Unknown names: Tertulla Longina (daughter)"
    },
    {
        "id": 103,
        "text": "AVITUS MARINUS CONIVGI CARISSIMAE FECIT",
        "description": "Unknown names: Avitus Marinus (husband)"
    }
]

# Save as JSON
with open('data/unknown_names.json', 'w', encoding='utf-8') as f:
    json.dump(unknown_name_inscriptions, f, indent=2, ensure_ascii=False)

print("✅ Created test data with unknown names:")
for insc in unknown_name_inscriptions:
    print(f"\n  ID {insc['id']}: {insc['text']}")
    print(f"  → {insc['description']}")

In [ ]:
# COMPARISON: Pattern-Only vs Hybrid Grammar Parser
print("🔬 COMPARING EXTRACTION METHODS")
print("="*80)
print("\nTest inscription: \"D M VIBIAE SABINAE FILIAE VIBIUS PAULUS PATER FECIT\"\n")

# Method 1: Pattern-Only (original)
print("📋 Method 1: Pattern-Only Extraction")
print("-" * 80)
from latinepi.parser import extract_entities
entities_pattern = extract_entities("D M VIBIAE SABINAE FILIAE VIBIUS PAULUS PATER FECIT")
if entities_pattern:
    for key, val in entities_pattern.items():
        print(f"  {key}: {val['value']} (confidence: {val['confidence']:.2f})")
else:
    print("  ❌ No entities extracted")

print(f"\n📊 Entities found: {len(entities_pattern)}")
print("\n⚠️  Problem: Unknown names like 'Vibia Sabina' and 'Vibius Paulus' were missed!\n")

# Method 2: Hybrid Grammar Parser
print("\n🧠 Method 2: Hybrid Grammar Parser")
print("-" * 80)
from latinepi.hybrid_parser import extract_entities_hybrid
entities_grammar = extract_entities_hybrid(
    "D M VIBIAE SABINAE FILIAE VIBIUS PAULUS PATER FECIT",
    use_morphology=False,
    use_dependencies=False,
    verbose=True
)

if entities_grammar:
    for key, val in entities_grammar.items():
        source = val.get("extraction_phase", "unknown")
        print(f"  {key}: {val['value']} (confidence: {val['confidence']:.2f}, source: {source})")
else:
    print("  No entities extracted")

print(f"\n📊 Entities found: {len(entities_grammar)}")
print("\n✅ Success: Grammar parser extracted all names including unknown ones!")
print("="*80)

In [ ]:
# Using the Hybrid Parser via CLI
print("🖥️  HYBRID PARSER VIA CLI")
print("="*60)
print("\nProcessing inscriptions with unknown names using --use-grammar flag:\n")

# Process with pattern-only (baseline)
!latinepi \
    --input data/unknown_names.json \
    --output output/unknown_pattern_only.json

print("\n" + "-"*60)
print("Pattern-only results:")
with open('output/unknown_pattern_only.json', 'r') as f:
    pattern_results = json.load(f)
    total_entities_pattern = sum(len([k for k in r.keys() if not k.endswith('_confidence') and k != 'inscription_id']) for r in pattern_results)
    print(f"  Total entities extracted: {total_entities_pattern}")

# Process with hybrid grammar parser
!latinepi \
    --input data/unknown_names.json \
    --output output/unknown_grammar.json \
    --use-grammar \
    --verbose

print("\n" + "-"*60)
print("Hybrid grammar parser results:")
with open('output/unknown_grammar.json', 'r') as f:
    grammar_results = json.load(f)
    total_entities_grammar = sum(len([k for k in r.keys() if not k.endswith('_confidence') and k != 'inscription_id']) for r in grammar_results)
    print(f"  Total entities extracted: {total_entities_grammar}")

print(f"\n✨ Improvement: {total_entities_grammar - total_entities_pattern} additional entities extracted!")

# Show detailed comparison
print("\n" + "="*60)
print("DETAILED COMPARISON")
print("="*60)
for i, (p_result, g_result) in enumerate(zip(pattern_results, grammar_results)):
    print(f"\n📜 Inscription {p_result.get('inscription_id')}:")
    print(f"  Pattern-only: {len([k for k in p_result.keys() if not k.endswith('_confidence') and k != 'inscription_id'])} entities")
    print(f"  With grammar: {len([k for k in g_result.keys() if not k.endswith('_confidence') and k != 'inscription_id'])} entities")

    # Show what grammar parser found that pattern didn't
    pattern_keys = set(k for k in p_result.keys() if not k.endswith('_confidence') and k != 'inscription_id')
    grammar_keys = set(k for k in g_result.keys() if not k.endswith('_confidence') and k != 'inscription_id')
    new_keys = grammar_keys - pattern_keys
    if new_keys:
        print(f"  ✅ Additional entities found by grammar parser:")
        for key in sorted(new_keys):
            print(f"     • {key}: {g_result[key]}")

## 7. EDH Single Inscription Download

Download a specific inscription from the Epigraphic Database Heidelberg.

In [ ]:
# Download inscription HD000001 from EDH
print("📥 Downloading inscription HD000001 from EDH...\n")

!latinepi \
    --download-edh HD000001 \
    --download-dir edh_downloads/

# Check what was downloaded
import os
edh_files = list(Path('edh_downloads').glob('*.json'))
print(f"\n✅ Downloaded {len(edh_files)} file(s) to edh_downloads/")

if edh_files:
    # Show structure of downloaded file
    with open(edh_files[0], 'r') as f:
        edh_data = json.load(f)

    print(f"\n📄 Downloaded file: {edh_files[0].name}")
    print(f"   Top-level keys: {list(edh_data.keys())}")

    # Show inscriptions if present
    if 'inscriptions' in edh_data:
        print(f"   Number of inscriptions: {len(edh_data['inscriptions'])}")
        if edh_data['inscriptions']:
            first_insc = edh_data['inscriptions'][0]
            print(f"   Inscription fields: {list(first_insc.keys())[:10]}...")

## 8. EDH Bulk Search and Download

Search for multiple inscriptions by criteria and download them in parallel.

⚠️ **Note**: This example uses small limits to avoid long download times. Adjust `--search-limit` for production use.

In [ ]:
# Search for inscriptions from Rome (modern findspot)
print("🔍 Searching EDH for inscriptions from Rome...\n")

!latinepi \
    --search-edh \
    --search-findspot-modern "rome*" \
    --search-limit 20 \
    --search-workers 5 \
    --download-dir edh_downloads/rome/

# Check results
rome_files = list(Path('edh_downloads/rome').glob('*.json'))
print(f"\n✅ Downloaded {len(rome_files)} inscriptions from Rome")
print(f"   Files saved to: edh_downloads/rome/")

# Show some inscription IDs
if rome_files:
    print(f"\n📋 Sample inscription IDs:")
    for f in rome_files[:5]:
        print(f"   - {f.stem}")

## 9. Temporal Search (By Date Range)

Search inscriptions by time period.

In [ ]:
# Search for 1st century AD inscriptions
print("🔍 Searching for 1st century AD inscriptions...\n")

!latinepi \
    --search-edh \
    --search-year-from 1 \
    --search-year-to 100 \
    --search-limit 15 \
    --download-dir edh_downloads/first_century/

# Check results
century_files = list(Path('edh_downloads/first_century').glob('*.json'))
print(f"\n✅ Downloaded {len(century_files)} inscriptions from 1st century AD")

## 10. Complete Pipeline: Search → Download → Extract → Analyze

Demonstrate the full workflow from search to analysis.

In [ ]:
# Step 1: Search and download inscriptions from a specific province
print("🔍 Step 1: Searching for inscriptions from Dalmatia...\n")

!latinepi \
    --search-edh \
    --search-province "Dalmatia" \
    --search-limit 10 \
    --download-dir edh_downloads/dalmatia/

# Step 2: Process all downloaded inscriptions
print("\n🔧 Step 2: Extracting entities from downloaded inscriptions...\n")

# Get list of downloaded files
dalmatia_files = list(Path('edh_downloads/dalmatia').glob('*.json'))

if dalmatia_files:
    # Process each file and collect results
    all_results = []

    for file_path in dalmatia_files:
        # For now, process files individually (in production you might batch this)
        output_file = f'output/dalmatia_{file_path.stem}.json'
        !latinepi \
            --input {str(file_path)} \
            --output {output_file} \
            --confidence-threshold 0.7

        with open(output_file, 'r') as f:
            results = json.load(f)
            all_results.extend(results)

    print(f"\n✅ Processed {len(dalmatia_files)} inscriptions")
    print(f"   Total entity records extracted: {len(all_results)}")

    # Save combined results
    with open('output/dalmatia_combined.json', 'w') as f:
        json.dump(all_results, f, indent=2, ensure_ascii=False)

    print(f"\n💾 Combined results saved to: output/dalmatia_combined.json")
else:
    print("⚠️  No files downloaded. The search may not have returned results.")

## 11. Data Analysis and Visualization

Analyze the extracted entities to gain insights.

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Load all extracted entities
with open('output/entities.json', 'r') as f:
    entities = json.load(f)

print("📊 ENTITY EXTRACTION ANALYSIS")
print("="*60 + "\n")

# Count entity types
entity_types = Counter()
confidence_scores = []

for record in entities:
    for key, value in record.items():
        if not key.endswith('_confidence') and key != 'inscription_id' and not key.endswith('_ambiguous'):
            entity_types[key] += 1
            confidence_key = f"{key}_confidence"
            if confidence_key in record:
                confidence_scores.append(record[confidence_key])

# Print statistics
print(f"Total inscriptions processed: {len(entities)}")
print(f"Total entities extracted: {sum(entity_types.values())}")
print(f"Average entities per inscription: {sum(entity_types.values()) / len(entities):.2f}")
print(f"\nMost common entity types:")
for entity_type, count in entity_types.most_common():
    print(f"  {entity_type}: {count}")

if confidence_scores:
    avg_confidence = sum(confidence_scores) / len(confidence_scores)
    print(f"\nAverage confidence score: {avg_confidence:.3f}")
    print(f"Confidence range: {min(confidence_scores):.3f} - {max(confidence_scores):.3f}")

# Visualization
if entity_types:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

    # Entity type distribution
    types, counts = zip(*entity_types.most_common())
    ax1.barh(types, counts, color='steelblue')
    ax1.set_xlabel('Count')
    ax1.set_title('Entity Type Distribution')
    ax1.invert_yaxis()

    # Confidence score distribution
    if confidence_scores:
        ax2.hist(confidence_scores, bins=20, color='coral', edgecolor='black')
        ax2.set_xlabel('Confidence Score')
        ax2.set_ylabel('Frequency')
        ax2.set_title('Confidence Score Distribution')
        ax2.axvline(avg_confidence, color='red', linestyle='--', label=f'Mean: {avg_confidence:.3f}')
        ax2.legend()

    plt.tight_layout()
    plt.savefig('output/analysis.png', dpi=150, bbox_inches='tight')
    print("\n📈 Visualizations saved to: output/analysis.png")
    plt.show()

## 12. Advanced Analysis: Name Patterns

Analyze Roman naming conventions in the extracted data.

In [ ]:
print("📊 ROMAN NAMING PATTERNS ANALYSIS")
print("="*60 + "\n")

# Count full name combinations
tria_nomina = 0  # praenomen + nomen + cognomen
duo_nomina = 0   # two of three
single_names = 0 # just one name

praenomina = Counter()
nomina = Counter()
cognomina = Counter()

for record in entities:
    has_praenomen = 'praenomen' in record
    has_nomen = 'nomen' in record
    has_cognomen = 'cognomen' in record

    name_count = sum([has_praenomen, has_nomen, has_cognomen])

    if name_count == 3:
        tria_nomina += 1
    elif name_count == 2:
        duo_nomina += 1
    elif name_count == 1:
        single_names += 1

    # Collect name components
    if has_praenomen:
        praenomina[record['praenomen']] += 1
    if has_nomen:
        nomina[record['nomen']] += 1
    if has_cognomen:
        cognomina[record['cognomen']] += 1

total_with_names = tria_nomina + duo_nomina + single_names

if total_with_names > 0:
    print(f"Naming Conventions:")
    print(f"  Tria nomina (3 names): {tria_nomina} ({tria_nomina/total_with_names*100:.1f}%)")
    print(f"  Duo nomina (2 names):  {duo_nomina} ({duo_nomina/total_with_names*100:.1f}%)")
    print(f"  Single names:          {single_names} ({single_names/total_with_names*100:.1f}%)")

    print(f"\nMost common praenomina:")
    for name, count in praenomina.most_common(5):
        print(f"  {name}: {count}")

    print(f"\nMost common nomina:")
    for name, count in nomina.most_common(5):
        print(f"  {name}: {count}")

    print(f"\nMost common cognomina:")
    for name, count in cognomina.most_common(5):
        print(f"  {name}: {count}")
else:
    print("⚠️  No name entities found in the data.")

## 13. Export Results for Further Analysis

Prepare data for external tools (Excel, R, etc.).

In [ ]:
# Convert all JSON results to CSV for spreadsheet analysis
print("💾 Exporting results to CSV format...\n")

# Load entities
with open('output/entities.json', 'r') as f:
    entities = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(entities)

# Save as CSV
df.to_csv('output/all_entities_export.csv', index=False)
print(f"✅ Exported {len(df)} records to: output/all_entities_export.csv")

# Create summary statistics CSV
summary_data = []
for col in df.columns:
    if not col.endswith('_confidence') and col != 'inscription_id':
        summary_data.append({
            'entity_type': col,
            'count': df[col].notna().sum(),
            'unique_values': df[col].nunique(),
            'avg_confidence': df[f"{col}_confidence"].mean() if f"{col}_confidence" in df.columns else None
        })

summary_df = pd.DataFrame(summary_data)
summary_df.to_csv('output/entity_summary.csv', index=False)
print(f"✅ Summary statistics saved to: output/entity_summary.csv")

# Display summary
print("\n📊 Entity Summary:")
print(summary_df.to_string(index=False))

print("\n" + "="*60)
print("📦 All outputs saved to 'output/' directory")
print("   Download these files to analyze in Excel, R, or other tools.")
print("="*60)

New Approach

In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 61.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
import spacy
import pandas as pd
import random
from spacy.tokens import DocBin, Span
from spacy.training.example import Example
from spacy.matcher import Matcher

def create_training_data_with_matcher(nlp, df):
    """
    Uses spaCy's Matcher to create robust, aligned training data.
    This is the proper way to create rule-based annotations for spaCy.
    """
    training_data = []

    # 1. Define the Matcher patterns for our entities
    # These patterns look for attributes of tokens, not raw text
    matcher = Matcher(nlp.vocab)

    # Pattern for names (e.g., 2-3 consecutive capitalized words)
    # {'IS_UPPER': True, 'OP': '+'} means "one or more uppercase tokens"
    name_pattern = [{"IS_UPPER": True, "IS_PUNCT": False}, {"IS_UPPER": True, "IS_PUNCT": False, "OP": "?"}, {"IS_UPPER": True, "IS_PUNCT": False, "OP": "?"}]

    # We can be more specific for deceased vs. dedicator later, but this is a good start
    matcher.add("PERSON_NAME", [name_pattern])

    # Pattern for age (e.g., the token 'ANNIS' followed by an uppercase token (the numeral))
    age_years_pattern = [{"LOWER": "annis"}, {"IS_UPPER": True}]
    matcher.add("AGE_YEARS", [age_years_pattern])

    # Pattern for occupation (simple keyword match)
    occupation_pattern = [{"LOWER": {"IN": ["medico", "pictori", "fabro", "veterano", "negotiatori", "centurioni"]}}]
    matcher.add("OCCUPATION", [occupation_pattern])

    # Pattern for relationships (simple keyword match)
    relationship_pattern = [{"LOWER": {"IN": ["coniunx", "pater", "mater", "filius", "filia", "frater"]}}]
    matcher.add("RELATIONSHIP", [relationship_pattern])

    # 2. Process each inscription text
    for _, row in df.iterrows():
        text = str(row['transcription']) # Use the expanded transcription
        doc = nlp(text)
        matches = matcher(doc)

        spans = [] # This will hold the Span objects

        for match_id, start_token, end_token in matches:
            label = nlp.vocab.strings[match_id]

            # Create a Span object from the matched tokens. This is the key step!
            # Spans are guaranteed to be aligned with token boundaries.
            span = Span(doc, start_token, end_token, label=label)

            # Specific logic to refine generic labels
            if label == "PERSON_NAME":
                # Heuristic: First name is likely the deceased, later names near a relationship are dedicators
                if "MANIBUS" in doc[0:start_token].text:
                    span.label_ = "DECEASED_NAME"
                elif "CONIUNX" in doc[start_token:end_token+2].text.upper() or "PATER" in doc[start_token:end_token+2].text.upper():
                    span.label_ = "DEDICATOR_NAME"
                else:
                    # Skip generic person names if we can't classify them
                    continue

            # For age, we only want to label the numeral, not the keyword
            if label == "AGE_YEARS":
                span = Span(doc, start_token + 1, end_token, label=label)

            spans.append(span)

        # Use spacy's utility to filter out overlapping spans, preferring longer ones
        filtered_spans = spacy.util.filter_spans(spans)

        if filtered_spans:
            training_data.append(Example.from_dict(doc, {"entities": [(s.start_char, s.end_char, s.label_) for s in filtered_spans]}))

    return training_data

def train_spacy_ner_from_csv(csv_path: str, model_path: str = "epigraphy_ner_model_v2", n_iter: int = 30):
    """
    Trains a spaCy NER model using a robust Matcher-based annotation approach.
    """
    print("--- Starting Model Training (v2 with Matcher) ---")

    nlp = spacy.blank("la") # Create a blank Latin model

    df = pd.read_csv(csv_path)
    print(f"Generating training data for {len(df)} inscriptions using Matcher...")

    # Generate the high-quality, aligned training data
    training_data = create_training_data_with_matcher(nlp, df)

    print(f"Generated {len(training_data)} valid training examples.")
    if not training_data:
        print("No training data generated. Check patterns and input text.")
        return

    # Create the DocBin object
    db = DocBin()
    for example in training_data:
        db.add(example.reference)

    # Save the processed data
    db.to_disk("./train.spacy")

    # Now, we can use the more modern spacy CLI training, or continue with the script method
    # For simplicity, we'll continue with the script method
    ner = nlp.add_pipe("ner")
    for example in training_data:
        for ent in example.reference.ents:
            ner.add_label(ent.label_)

    print("\n--- Begin Training Loop ---")
    optimizer = nlp.begin_training()

    for itn in range(n_iter):
        random.shuffle(training_data)
        losses = {}

        for example in training_data:
            nlp.update([example], drop=0.5, sgd=optimizer, losses=losses)

        print(f"Iteration {itn + 1}/{n_iter}  |  Losses: {losses}")

    nlp.to_disk(model_path)
    print(f"\n--- Training Complete ---")
    print(f"Model saved to '{model_path}'")
    return model_path

# --- Example Usage (Updated) ---
if __name__ == '__main__':
    csv_filename = "data/fake-epigraphs.csv"
    # Create the dummy CSV again to ensure it's available
    # dummy_data = {
    #     'id': [701, 702, 707, 719, 285],
    #     'text': [...], # Same as before
    #     'transcription': [
    #         'DIS MANIBUS GAIO IULIO VALENTI MILITI VETERANO LEGIONIS X GEMINAE VIXIT ANNIS XXXV IULIA SECUNDINA CONIUNX FECIT',
    #         'DIS MANIBUS MARCO PETRONIO LONGO VETERANO LEGIONIS II AUGUSTAE FLAVIA PROCULA CONIUNX ET MARCUS PETRONIUS FILIUS FACIENDUM CURAVERUNT',
    #         'DIS MANIBUS QUINTO HOSTILIO MARCELLO NEGOTIATORI VINARIO VIXIT ANNIS LX HOSTILIA FILIA PATRI OPTIMO',
    #         'DIS MANIBUS VALERIO VITALI PICTORI VIXIT ANNIS XXXVIIII VALERIA MAXIMINA CONIUNX MARITO OPTIMO',
    #         'DIS MANIBUS MARCO AURELIO SEVERO ALEXANDRO. VIXIT ANNIS XXVI'
    #     ]
    # }
   # pd.DataFrame(dummy_data).to_csv(csv_filename, index=False)

    # Train the new, more robust model
    model_path = train_spacy_ner_from_csv(csv_path=csv_filename, n_iter=25)

    # Load and test the newly trained model
    print("\n--- Testing the Trained Model (v2) ---")
    nlp_trained = spacy.load(model_path)

    # Use the same test inscription as before
    test_inscription = "DIS MANIBUS TITO FLAVIO MARTIALI MEDICO VIXIT ANNIS LII FLAVIA PRISCA CONIUNX FECIT"
    doc = nlp_trained(test_inscription)

    print(f"Processing text: '{doc.text}'")
    print("Entities found:")
    if not doc.ents:
        print("No entities found.")
    else:
        for ent in doc.ents:
            print(f"- Text: '{ent.text}', Label: '{ent.label_}'")

--- Starting Model Training (v2 with Matcher) ---
Generating training data for 800 inscriptions using Matcher...
Generated 63 valid training examples.

--- Begin Training Loop ---
Iteration 1/25  |  Losses: {'ner': np.float32(277.468)}
Iteration 2/25  |  Losses: {'ner': np.float32(104.55537)}
Iteration 3/25  |  Losses: {'ner': np.float32(75.98132)}
Iteration 4/25  |  Losses: {'ner': np.float32(40.778038)}
Iteration 5/25  |  Losses: {'ner': np.float32(28.674349)}
Iteration 6/25  |  Losses: {'ner': np.float32(25.62096)}
Iteration 7/25  |  Losses: {'ner': np.float32(20.355614)}
Iteration 8/25  |  Losses: {'ner': np.float32(10.6596775)}
Iteration 9/25  |  Losses: {'ner': np.float32(9.254774)}
Iteration 10/25  |  Losses: {'ner': np.float32(7.9641757)}
Iteration 11/25  |  Losses: {'ner': np.float32(9.416325)}
Iteration 12/25  |  Losses: {'ner': np.float32(5.1306725)}
Iteration 13/25  |  Losses: {'ner': np.float32(5.645845)}
Iteration 14/25  |  Losses: {'ner': np.float32(3.9651623)}
Iteration

In [18]:
import spacy
import pandas as pd
import json

# --- Step 1: Helper Functions for Data Cleaning and Structuring ---

def roman_to_int(s: str) -> int:
    """Converts a Roman numeral string to an integer."""
    roman_map = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
    if not s or not all(c in roman_map for c in s):
        return 0 # Return 0 for invalid or empty strings

    result = 0
    for i in range(len(s)):
        # Check for subtractive notation (e.g., IV, IX)
        if i > 0 and roman_map[s[i]] > roman_map[s[i-1]]:
            result += roman_map[s[i]] - 2 * roman_map[s[i-1]]
        else:
            result += roman_map[s[i]]
    return result

def parse_roman_name(name_str: str) -> dict:
    """Parses a full Roman name string into its components."""
    parts = name_str.strip().split()
    name_dict = {"praenomen": None, "nomen": None, "cognomen": None, "full_name": name_str}

    # A simple parser based on the number of name parts
    if len(parts) == 1:
        name_dict["cognomen"] = parts[0] # Often just the cognomen is used
    elif len(parts) == 2:
        name_dict["nomen"] = parts[0]
        name_dict["cognomen"] = parts[1]
    elif len(parts) >= 3:
        name_dict["praenomen"] = parts[0]
        name_dict["nomen"] = parts[1]
        name_dict["cognomen"] = " ".join(parts[2:]) # Handle multiple cognomina

    return name_dict

# --- Step 2: The Main Extraction Function ---

def extract_structured_data(inscriptions: list[str], model_path: str) -> list[dict]:
    """
    Applies a trained spaCy model to extract structured data from unseen inscriptions.

    Args:
        inscriptions (list[str]): A list of inscription texts to process.
        model_path (str): The directory path of the trained spaCy model.

    Returns:
        list[dict]: A list of dictionaries, where each dictionary represents the
                    structured data extracted from one inscription.
    """
    print(f"--- Loading model from '{model_path}' ---")
    try:
        nlp = spacy.load(model_path)
    except IOError:
        print(f"Error: Could not load model from '{model_path}'.")
        print("Please ensure the model path is correct and the model is trained.")
        return []

    print("--- Processing inscriptions ---")
    structured_results = []

    for text in inscriptions:
        # We process the uppercase version for consistency with training
        doc = nlp(text.upper())

        # Initialize the structured data dictionary for this inscription
        result = {
            "source_text": text,
            "deceased": {"full_name": None, "praenomen": None, "nomen": None, "cognomen": None},
            "age_at_death": {"years": None, "months": None, "days": None},
            "career": {"occupation": None, "military_unit": None},
            "dedicators": [] # Using a list as there can be multiple
        }

        # This will hold the last dedicator found, to link them with a relationship
        last_dedicator = None

        for ent in doc.ents:
            # Process each entity and place it in the correct part of the dictionary
            if ent.label_ == "DECEASED_NAME":
                result["deceased"] = parse_roman_name(ent.text)

            elif ent.label_ == "DEDICATOR_NAME":
                dedicator_info = parse_roman_name(ent.text)
                dedicator_info["relationship"] = "Unknown" # Default value
                result["dedicators"].append(dedicator_info)
                last_dedicator = dedicator_info # Keep track of the last one found

            elif ent.label_ == "RELATIONSHIP":
                 # Heuristic: Assume the relationship applies to the last dedicator found
                if last_dedicator:
                    last_dedicator["relationship"] = ent.text

            elif ent.label_ == "AGE_YEARS":
                result["age_at_death"]["years"] = roman_to_int(ent.text)
            elif ent.label_ == "AGE_MONTHS":
                result["age_at_death"]["months"] = roman_to_int(ent.text)
            elif ent.label_ == "AGE_DAYS":
                result["age_at_death"]["days"] = roman_to_int(ent.text)

            elif ent.label_ == "OCCUPATION":
                result["career"]["occupation"] = ent.text
            elif ent.label_ == "MILITARY_UNIT":
                result["career"]["military_unit"] = ent.text

        structured_results.append(result)

    print(f"--- Extraction complete for {len(inscriptions)} inscriptions ---")
    return structured_results

# --- Step 3: Example Usage ---

if __name__ == '__main__':
    # Define the path to the model we trained in the previous step
    trained_model_path = "epigraphy_ner_model_v2"

    # A list of new, unseen inscriptions to test the model on
    unseen_inscriptions = [
        # Test case 1: A simple inscription with a dedicator
        "DIS MANIBUS AURELIAE SATURNINAE VIXIT ANNIS XXXI AURELIUS FELIX CONIUNX BENE MERENTI POSUIT",

        # Test case 2: A soldier with a military unit and age
        "D M T CLAUDIO MAXIMO VETERANO LEGIONIS VII GEMINAE VIXIT ANNIS LXII MENSIBVS II",

        # Test case 3: An inscription with an occupation but no dedicator
        "DIS MANIBUS GAIO APULEIO DIOCLI PICTORI QUI VIXIT ANNIS QUADRAGINTA", # Note: a slightly different phrasing

        # Test case 4: A child's epitaph with a complex age
        "D M S FLAVIAE IANUARIAE VIXIT ANNIS V MENSIBVS VIII DIEBVS X PATER ET MATER FILIAE DULCISSIMAE",

        # Test case 5 (edge case): An inscription the model might struggle with
        "MEMORIAE AETERNAE VALERIAE FRONTINAE"
    ]

    # Run the extraction function
    extracted_data = extract_structured_data(unseen_inscriptions, trained_model_path)

    # Print the results in a clean, readable JSON format
    if extracted_data:
        print("\n--- Extracted Data Results ---")
        for i, data in enumerate(extracted_data):
            print(f"\n--- Inscription #{i+1} ---")
            print(json.dumps(data, indent=2))

--- Loading model from 'epigraphy_ner_model_v2' ---
--- Processing inscriptions ---
--- Extraction complete for 5 inscriptions ---

--- Extracted Data Results ---

--- Inscription #1 ---
{
  "source_text": "DIS MANIBUS AURELIAE SATURNINAE VIXIT ANNIS XXXI AURELIUS FELIX CONIUNX BENE MERENTI POSUIT",
  "deceased": {
    "full_name": null,
    "praenomen": null,
    "nomen": null,
    "cognomen": null
  },
  "age_at_death": {
    "years": null,
    "months": null,
    "days": null
  },
  "career": {
    "occupation": null,
    "military_unit": null
  },
  "dedicators": []
}

--- Inscription #2 ---
{
  "source_text": "D M T CLAUDIO MAXIMO VETERANO LEGIONIS VII GEMINAE VIXIT ANNIS LXII MENSIBVS II",
  "deceased": {
    "full_name": null,
    "praenomen": null,
    "nomen": null,
    "cognomen": null
  },
  "age_at_death": {
    "years": null,
    "months": null,
    "days": null
  },
  "career": {
    "occupation": "VETERANO",
    "military_unit": null
  },
  "dedicators": []
}

--- Inscri